In [108]:
import numpy as np

In [109]:
#2a
def func(x):
    return np.e ** (x[0] + 3 * x[1]) + np.e ** (x[0] - 3 * x[1]) + np.e ** (-x[0])

def f_grad(x):
    return np.array([np.e ** (x[0] + 3 * x[1]) + np.e ** (x[0] - 3 * x[1]) - np.e ** (-x[0]), 
                     3 * (np.e ** (6 * x[1]) - 1) * np.e ** (x[0] - 3 * x[1])])

In [110]:
#2b
eps = 0.01
eps_grid = 0.01
x1 = -1
max_M = -1000000000

#random
iters = 1000000
for i in range(iters):
    p = np.random.random([2]) * 2 - 1
    p000 = np.array([p[0] + eps, p[1] + eps])
    p001 = np.array([p[0] + eps, p[1] - eps])
    p010 = np.array([p[0] - eps, p[1] + eps])
    p011 = np.array([p[0] - eps, p[1] - eps])
    p100 = np.array([p[0], p[1] + eps])
    p101 = np.array([p[0], p[1] - eps])
    p110 = np.array([p[0] + eps, p[1]])
    p111 = np.array([p[0] - eps, p[1]])
    max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p000)) / np.linalg.norm(p - p000))
    max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p001)) / np.linalg.norm(p - p001))
    max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p010)) / np.linalg.norm(p - p010))
    max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p011)) / np.linalg.norm(p - p011))
    max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p100)) / np.linalg.norm(p - p100))
    max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p101)) / np.linalg.norm(p - p101))
    max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p110)) / np.linalg.norm(p - p110))
    max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p111)) / np.linalg.norm(p - p111))
print(max_M) # approx 523.1495449078291

#grid
while x1 <= 1:
    y1 = -1
    while y1 <= 1:
        p = np.array([x1, y1])
        p000 = np.array([p[0] + eps, p[1] + eps])
        p001 = np.array([p[0] + eps, p[1] - eps])
        p010 = np.array([p[0] - eps, p[1] + eps])
        p011 = np.array([p[0] - eps, p[1] - eps])
        p100 = np.array([p[0], p[1] + eps])
        p101 = np.array([p[0], p[1] - eps])
        p110 = np.array([p[0] + eps, p[1]])
        p111 = np.array([p[0] - eps, p[1]])
        max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p000)) / np.linalg.norm(p - p000))
        max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p001)) / np.linalg.norm(p - p001))
        max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p010)) / np.linalg.norm(p - p010))
        max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p011)) / np.linalg.norm(p - p011))
        max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p100)) / np.linalg.norm(p - p100))
        max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p101)) / np.linalg.norm(p - p101))
        max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p110)) / np.linalg.norm(p - p110))
        max_M = max(max_M, np.linalg.norm(f_grad(p) - f_grad(p111)) / np.linalg.norm(p - p111))
        y1 += eps_grid
    x1 += eps_grid
print(max_M) # хуже рандома

#можно посчитать корень из суммы квадратов 2 производной и подставить:
#f'_xx = f = e^(x+3y)+e^(x-3y)+e^(-x) ; f'_xy = 3(e^(x+3y)-e^(x-3y)); f'_yy = 9(e^(x+3y)+e^(x-3y)).
# все степени не более 4
# в f'_xx коэф перед e^4 не более 3
# в 2 * f'_xy коэф перед e^4 не более 2 * 3 = 6
# в f'_yy коэф перед e^4 не более 9 * (1 + 1) = 18
# получается оценка e^4 * sqrt(3 ** 2 + 6 ** 2 + 18 ** 2) = 1091
# рандом выдает 523 значит оценка точна весьма

523.4107956871724
523.4107956871724


In [111]:
#2c
my_max_M = 1091
start_point = np.array([-0.5, -0.5])
steps_num = 3000

#optimal step
alpha = 1.0 / (my_max_M)
traj_opt_step = []
x_start = start_point
traj_opt_step.append(x_start.copy())
cur_x = x_start.copy()
for i in range(steps_num):
    cur_x = cur_x - alpha * f_grad(cur_x)
    traj_opt_step.append(cur_x.copy()) 

In [112]:
print(traj_opt_step[-1])

[-3.46664006e-01 -5.39809155e-16]


In [113]:
#nesterov
a = 1.0 / my_max_M
traj_nesterov = [start_point.copy()]
cx = start_point.copy()
cy = start_point.copy()
for i in range(steps_num):
    px = cx
    py = cy
    pa = a
    cx = cy - (f_grad(cy) / my_max_M)
    a = (- pa ** 2 + np.sqrt(pa ** 4 + 4 * pa ** 2)) / 2
    cb = pa * (1 - pa) / (pa ** 2 + a)
    cy = cx + cb * (cx - px)
    traj_nesterov.append(cx.copy()) 

In [114]:
print(traj_nesterov[-1])

[-3.47055191e-01  1.18742540e-09]
